<a href="https://colab.research.google.com/github/sahil1239/Fall-Prediction-ELC/blob/main/Model%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import os
import pandas as pd
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses
import pickle
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
import pandas as pd

path = "./"

# Load the training data from CSV files
train_x = pd.read_csv(path + 'train_x.csv')
train_y = pd.read_csv(path + 'train_y.csv')
test_x = pd.read_csv(path + 'test_x.csv')
test_y = pd.read_csv(path + 'test_y.csv')

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)


In [ ]:
unique_values, counts = np.unique(train_y, return_counts=True)
print(unique_values, counts)

In [ ]:
unique_values, counts = np.unique(test_y, return_counts=True)
print(unique_values, counts)

In [ ]:
n_timesteps = 50
n_features = 9
n_outputs = 1

def hybrid_model():

    model = models.Sequential()

    # First CNN layer

    model.add(layers.Conv1D(kernel_size=5,
                            filters=64,
                            activation='relu',
                            input_shape=(n_timesteps, n_features)))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling1D(3, strides=2, padding="valid"))

    # Second CNN layer

    model.add(layers.Conv1D(kernel_size=5,
                            filters=64,
                            activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling1D(3, strides=2, padding="valid"))

    # Third CNN layer

    model.add(layers.Conv1D(kernel_size=5,
                            filters=64,
                            activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling1D(3, strides=2, padding="valid"))

    model.add(layers.Dense(64))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))

    # First LSTM
    model.add(layers.LSTM(64, return_sequences=True))
    model.add(layers.Dropout(0.5))

    # Second LSTM
    model.add(layers.LSTM(64, return_sequences=True))
    model.add(layers.Dropout(0.5))

    model.add(layers.Flatten())
    model.add(layers.Dense(n_outputs, activation='sigmoid'))


    print(model.summary())


    opt = optimizers.Adam(learning_rate=0.0005)
    lss = losses.BinaryFocalCrossentropy()
    model.compile(loss=lss, optimizer=opt, metrics=['accuracy'])

    return model

In [ ]:
n_epochs = 200
model = hybrid_model()
history = model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs = n_epochs, batch_size = 64)

In [ ]:
y_pred =np.squeeze(model.predict(test_x))

In [ ]:
for i in range(len(y_pred)):
  if y_pred[i]>0.5:
    y_pred[i]=1
  else:
    y_pred[i]=0

In [ ]:
unique_values, counts = np.unique(test_y, return_counts=True)
print(unique_values, counts)

In [ ]:
unique_values, counts = np.unique(y_pred, return_counts=True)
print(unique_values, counts)

In [ ]:
print(classification_report(test_y, y_pred, target_names=['no fall', 'fall']))

In [ ]:
print(confusion_matrix(test_y, y_pred))

In [ ]:
model.save("model.h5")

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

modified_data = './data_File.csv'

# Read the CSV file
df = pd.read_csv(modified_data)

# Separate the features (X) and the target variable (y)
X = df.drop(['Output'], axis=1).values
y = df['Output'].values

# Split the data into train and test sets, stratifying based on the 'Output' column
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Print the train and test set shapes to verify the proportions
print("Train Set - Not Fall: {}, Fall: {}".format(np.sum(y_train == 0), np.sum(y_train == 1)))
print("Test Set - Not Fall: {}, Fall: {}".format(np.sum(y_test == 0), np.sum(y_test == 1)))


In [ ]:
import numpy as np
import pandas as pd
# importing file in df
df = pd.read_csv('./data_File.csv')

x = np.array(df.drop(['Output'], axis=1))
y = np.array(df['Output'])

group_size = 50
num_groups = len(df) // group_size

inputs = []
outputs = []

for i in range(num_groups):
    start_index = i * group_size
    end_index = (i + 1) * group_size

    inp = x[start_index:end_index]
    unique_values, counts = np.unique(y[start_index:end_index], return_counts=True)
    majority_element = unique_values[np.argmax(counts)]

    inputs.append(inp)
    outputs.append(majority_element)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Combine inputs and outputs into a single DataFrame
df_combined = pd.DataFrame({'Inputs': inputs, 'Outputs': outputs})

# Split the combined DataFrame into train and test sets
train_df, test_df = train_test_split(df_combined, test_size=0.2, random_state=42)

# Split the train and test sets into input and output DataFrames
train_x_df = pd.concat([train_df['Inputs']], ignore_index=True)
train_y_df = pd.DataFrame(train_df['Outputs'])
test_x_df = pd.concat([test_df['Inputs']], ignore_index=True)
test_y_df = pd.DataFrame(test_df['Outputs'])

# Save the train and test sets to separate CSV files
train_x_df.to_csv('train_x.csv', index=False)
train_y_df.to_csv('train_y.csv', index=False)
test_x_df.to_csv('test_x.csv', index=False)
test_y_df.to_csv('test_y.csv', index=False)